# Data Cleaning & Feature Engineering

**Notebook:** `data_cleaning_and_feature_engineering.ipynb`

**Purpose:** This notebook prepares the Kaggle-sourced loan dataset for modeling. It contains well-documented, professional explanations for each transformation so reviewers and collaborators understand the reasoning behind each step. The notebook performs:

- Data loading and schema inspection
- Data cleaning and normalization
- Parsing and type conversions for percent strings
- Domain-driven feature engineering (repayment velocity, credit utilization, credit age, delinquency flags)
- Categorical encoding and numerical scaling
- Final dataset validation and export

**How to use:** Edit the `RAW_PATH` at the top to point to the CSV you have (default uses `loan_data_final.csv`). Run every cell top-to-bottom. Each code cell is accompanied by a descriptive explanation for auditors and stakeholders.

---


In [1]:
# 0. Setup: imports and file paths
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os

RAW_PATH = '/content/loan_data_final.csv'   # change if using a different file
OUTPUT_PATH = '/content/loan_data_final_output.csv'

print('Raw path:', RAW_PATH)
print('Output path:', OUTPUT_PATH)

Raw path: /content/loan_data_final.csv
Output path: /content/loan_data_final_output.csv


### Explanation for Cell : 0. Setup: imports and file paths
This cell imports necessary libraries (`pandas`, `numpy`, `StandardScaler`, `os`) and defines file paths for the raw input CSV (`RAW_PATH`) and the processed output CSV (`OUTPUT_PATH`).

**Output:** It prints the defined `RAW_PATH` and `OUTPUT_PATH`.
`Raw path: /content/loan_data_final.csv`
`Output path: /content/loan_data_final_output.csv`

In [2]:
# 1. Load dataset
df = pd.read_csv(RAW_PATH)
print('Initial shape:', df.shape)
df.head()

Initial shape: (9578, 14)


,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,all_other,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,all_other,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


### Explanation for Cell : 1. Load dataset
This cell loads the CSV file specified by `RAW_PATH` into a pandas DataFrame named `df`. It then prints the initial shape of the DataFrame and displays the first 5 rows using `df.head()`.

**Output:**
`Initial shape: (9578, 14)`
It then displays a table showing the first 5 rows and all 14 columns of the loaded dataset, giving a glimpse of the data structure and content.

In [3]:
# 2. Quick inspection
print('Columns:', df.columns.tolist())
print('\nInfo:')
display(df.info())
print('\nMissing values per column:')
print(df.isnull().sum())

Columns: ['credit.policy', 'purpose', 'int.rate', 'installment', 'log.annual.inc', 'dti', 'fico', 'days.with.cr.line', 'revol.bal', 'revol.util', 'inq.last.6mths', 'delinq.2yrs', 'pub.rec', 'not.fully.paid']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit.policy      9578 non-null   int64  
 1   purpose            9578 non-null   object 
 2   int.rate           9578 non-null   float64
 3   installment        9578 non-null   float64
 4   log.annual.inc     9578 non-null   float64
 5   dti                9578 non-null   float64
 6   fico               9578 non-null   int64  
 7   days.with.cr.line  9578 non-null   float64
 8   revol.bal          9578 non-null   int64  
 9   revol.util         9578 non-null   float64
 10  inq.last.6mths     9578 non-null   int64  
 11  delinq.2yrs        9578 non-null   int64  
 12  p

None


Missing values per column:
credit.policy        0
purpose              0
int.rate             0
installment          0
log.annual.inc       0
dti                  0
fico                 0
days.with.cr.line    0
revol.bal            0
revol.util           0
inq.last.6mths       0
delinq.2yrs          0
pub.rec              0
not.fully.paid       0
dtype: int64


### Explanation for Cell : 2. Quick inspection
This cell performs a quick inspection of the DataFrame `df`. It prints the list of column names, displays a summary of the DataFrame's information including non-null counts and data types using `df.info()`, and checks for missing values in each column using `df.isnull().sum()`.

**Output:**
`Columns: ['credit.policy', 'purpose', 'int.rate', 'installment', 'log.annual.inc', 'dti', 'fico', 'days.with.cr.line', 'revol.bal', 'revol.util', 'inq.last.6mths', 'delinq.2yrs', 'pub.rec', 'not.fully.paid']`
It then displays the `df.info()` output, showing 9578 entries, 14 columns, their non-null counts, and data types (6 float64, 7 int64, 1 object). It also confirms:
`Missing values per column:`
(All columns show 0 missing values)

In [4]:
# 3. Basic cleaning & normalize column names
# drop exact duplicates
df = df.drop_duplicates().reset_index(drop=True)

# normalize column names
df.columns = [c.strip().lower().replace(' ', '_').replace('.', '_') for c in df.columns]

# view first rows and shape
print('After dedup & rename shape:', df.shape)
df.head()

After dedup & rename shape: (9578, 14)


,credit_policy,purpose,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec,not_fully_paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,all_other,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,all_other,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


### Explanation for Cell : 3. Basic cleaning & normalize column names
This cell performs basic data cleaning: it drops exact duplicate rows from the DataFrame and then normalizes the column names by converting them to lowercase, replacing spaces and periods with underscores, and stripping leading/trailing whitespace. Finally, it prints the shape of the DataFrame after these operations and shows the first 5 rows with the new column names.

**Output:**
`After dedup & rename shape: (9578, 14)`
It displays the first 5 rows of the DataFrame, similar to before, but with the cleaned and normalized column names like `credit_policy`, `int_rate`, `days_with_cr_line`, etc.

In [5]:
# 4. Ensure numeric columns are numeric (coerce if needed)
num_cols_guess = ['int_rate','installment','log_annual_inc','dti','fico','days_with_cr_line','days_with_credit_line',
                  'revol_bal','revol_util','inq_last_6mths','delinq_2yrs','pub_rec','public_rec']

# Keep only those present
num_cols = [c for c in num_cols_guess if c in df.columns]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

print('Numeric columns coerced:', num_cols)
print(df[num_cols].dtypes)

Numeric columns coerced: ['int_rate', 'installment', 'log_annual_inc', 'dti', 'fico', 'days_with_cr_line', 'revol_bal', 'revol_util', 'inq_last_6mths', 'delinq_2yrs', 'pub_rec']
int_rate             float64
installment          float64
log_annual_inc       float64
dti                  float64
fico                   int64
days_with_cr_line    float64
revol_bal              int64
revol_util           float64
inq_last_6mths         int64
delinq_2yrs            int64
pub_rec                int64
dtype: object


### Explanation for Cell : 4. Ensure numeric columns are numeric (coerce if needed)
This cell identifies a list of potentially numeric columns. It then iterates through these columns, and if they exist in the DataFrame, it attempts to convert them to numeric types using `pd.to_numeric` with `errors='coerce'`, which will turn any non-convertible values into `NaN`. Finally, it prints the list of columns that were coerced and their resulting data types.

**Output:**
`Numeric columns coerced: ['int_rate', 'installment', 'log_annual_inc', 'dti', 'fico', 'days_with_cr_line', 'revol_bal', 'revol_util', 'inq_last_6mths', 'delinq_2yrs', 'pub_rec']`
It then lists the `dtype` for these columns, confirming they are `float64` or `int64`.

In [6]:
# 5. Parse percent strings (interest rate, revol_util) if present
eps = 1e-9

if 'int_rate' in df.columns:
    # remove % and convert to fraction
    df['int_rate'] = df['int_rate'].astype(str).str.replace('%','').str.strip()
    df['int_rate'] = pd.to_numeric(df['int_rate'], errors='coerce')/100.0

if 'revol_util' in df.columns:
    df['revol_util'] = df['revol_util'].astype(str).str.replace('%','').str.strip()
    df['revol_util'] = pd.to_numeric(df['revol_util'], errors='coerce')/100.0

print('int_rate NA count:', df.get('int_rate', pd.Series()).isnull().sum())
print('revol_util NA count:', df.get('revol_util', pd.Series()).isnull().sum())

int_rate NA count: 0
revol_util NA count: 0


### Explanation for Cell : 5. Parse percent strings (interest rate, revol_util) if present
This cell checks for `int_rate` and `revol_util` columns. If found, it cleans them by removing '%' symbols, stripping whitespace, and converting them to numeric (float) values, dividing by 100 to represent them as fractions. `errors='coerce'` is used to handle potential conversion issues. It then prints the count of NaN values for these columns.

**Output:**
`int_rate NA count: 0`
`revol_util NA count: 0`
This indicates that after the conversion, there are no missing values in these columns.

In [7]:
# 6. Derived features
# repayment_velocity_proxy: proxy using fico & dti
if ('fico' in df.columns) and ('dti' in df.columns):
    df['repayment_velocity_proxy'] = (df['fico'] / 850.0) / (df['dti'] + eps)
else:
    df['repayment_velocity_proxy'] = np.nan

# credit_utilization: prefer direct if credit limit available; else approximate from revol_util
if ('revol_bal' in df.columns) and ('revol_util' in df.columns):
    df['approx_credit_limit'] = np.where((df['revol_util']>0) & df['revol_bal'].notnull(),
                                        df['revol_bal'] / (df['revol_util'] + eps),
                                        np.nan)
    df['credit_utilization'] = df['revol_bal'] / (df['approx_credit_limit'] + eps)
else:
    df['credit_utilization'] = np.nan

# debt_to_income_calc if log_annual_inc exists
if 'debt_to_income' not in df.columns and 'log_annual_inc' in df.columns:
    df['annual_inc'] = np.exp(df['log_annual_inc'])
    df['debt_to_income_calc'] = df['installment'] / (df['annual_inc']/12 + eps)

# credit age years
if 'days_with_cr_line' in df.columns:
    df['credit_age_years'] = df['days_with_cr_line'] / 365.0
elif 'days_with_credit_line' in df.columns:
    df['credit_age_years'] = df['days_with_credit_line'] / 365.0

# delinquency flags
if 'inq_last_6mths' in df.columns:
    df['has_recent_inq'] = (df['inq_last_6mths'] > 0).astype(int)
if 'delinq_2yrs' in df.columns:
    df['has_past_delinquency'] = (df['delinq_2yrs'] > 0).astype(int)

print('Derived features created. Columns now include:', [c for c in df.columns if c.startswith('repayment') or c.startswith('credit_') or c.startswith('debt_to_income') or c.endswith('years') or c.startswith('has_')][:20])

Derived features created. Columns now include: ['credit_policy', 'repayment_velocity_proxy', 'credit_utilization', 'debt_to_income_calc', 'credit_age_years', 'has_recent_inq', 'has_past_delinquency']


### Explanation for Cell : 6. Derived features
This cell engineers several new features based on existing columns:
*   `repayment_velocity_proxy`: Calculated from `fico` and `dti`.
*   `approx_credit_limit` and `credit_utilization`: Calculated from `revol_bal` and `revol_util`.
*   `annual_inc` and `debt_to_income_calc`: `annual_inc` is derived by exponentiating `log_annual_inc`, and `debt_to_income_calc` is derived from `installment` and `annual_inc`.
*   `credit_age_years`: Converts `days_with_cr_line` to years.
*   `has_recent_inq` and `has_past_delinquency`: Binary flags based on `inq_last_6mths` and `delinq_2yrs`.
It then prints a list of the newly created derived features.

**Output:**
`Derived features created. Columns now include: ['credit_policy', 'repayment_velocity_proxy', 'credit_utilization', 'debt_to_income_calc', 'credit_age_years', 'has_recent_inq', 'has_past_delinquency']`

In [8]:
# 7. Target handling - create binary target 'target_default' if possible
possible_targets = [c for c in ['not_fully_paid','loan_status','default_flag','default','is_default'] if c in df.columns]
if possible_targets:
    tgt = possible_targets[0]
    print('Detected target column:', tgt)
    df['target_default'] = df[tgt].apply(lambda x: 1 if str(x).strip().lower() in ['1','yes','y','true','t'] else 0)
else:
    print('No obvious target column detected. Please set one manually.')

print('Target distribution (if created):')
if 'target_default' in df.columns:
    print(df['target_default'].value_counts(dropna=False))

Detected target column: not_fully_paid
Target distribution (if created):
target_default
0    8045
1    1533
Name: count, dtype: int64


### Explanation for Cell : 7. Target handling - create binary target 'target_default' if possible
This cell attempts to identify a target column (like `not_fully_paid`, `loan_status`, etc.) from a predefined list. If a suitable target column is found, it creates a new binary column `target_default` (1 for default, 0 otherwise). It then prints the detected target column and the value distribution of the `target_default` column.

**Output:**
`Detected target column: not_fully_paid`
`Target distribution (if created):`
`target_default`
`0    8045`
`1    1533`
`Name: count, dtype: int64`
This shows that `not_fully_paid` was identified as the target, and its distribution is 8045 non-defaults and 1533 defaults.

In [9]:
# 8. Categorical encoding - simple one-hot for 'purpose' if present
if 'purpose' in df.columns:
    n_unique = df['purpose'].nunique()
    print('Purpose unique count:', n_unique)
    if n_unique <= 20:
        dummies = pd.get_dummies(df['purpose'], prefix='purpose', drop_first=True)
        df = pd.concat([df, dummies], axis=1)
        df.drop('purpose', axis=1, inplace=True)
    else:
        print('High cardinality for purpose - consider target or frequency encoding instead.')
else:
    print('No purpose column to encode.')

print('Shape after encoding:', df.shape)

Purpose unique count: 4
Shape after encoding: (9578, 25)


### Explanation for Cell : 8. Categorical encoding - simple one-hot for 'purpose' if present
This cell handles categorical encoding for the 'purpose' column if it exists. It checks the number of unique values in 'purpose'. If there are 20 or fewer unique values, it performs one-hot encoding using `pd.get_dummies`, creating new columns for each purpose category and dropping the original 'purpose' column. If there are too many unique values, it suggests alternative encoding methods.

**Output:**
`Purpose unique count: 4`
`Shape after encoding: (9578, 25)`
This indicates that 'purpose' had 4 unique categories and was successfully one-hot encoded, increasing the total number of columns in the DataFrame to 25.

In [10]:
# 9. Scaling numeric features (StandardScaler)
from sklearn.impute import SimpleImputer
scale_cols = ['installment','dti','fico','revol_bal','revol_util','repayment_velocity_proxy','credit_utilization','debt_to_income_calc','credit_age_years']
scale_cols = [c for c in scale_cols if c in df.columns]
print('Scaling columns:', scale_cols)

# Simple impute then scale
if scale_cols:
    imputer = SimpleImputer(strategy='median')
    df[scale_cols] = imputer.fit_transform(df[scale_cols])
    scaler = StandardScaler()
    df[scale_cols] = scaler.fit_transform(df[scale_cols])

print('Scaling complete.')

Scaling columns: ['installment', 'dti', 'fico', 'revol_bal', 'revol_util', 'repayment_velocity_proxy', 'credit_utilization', 'debt_to_income_calc', 'credit_age_years']
Scaling complete.


### Explanation for Cell : 9. Scaling numeric features (StandardScaler)
This cell identifies a list of numeric features to be scaled. It first imputes any missing values in these columns using `SimpleImputer` with a 'median' strategy. Then, it applies `StandardScaler` to these columns, transforming them to have a mean of 0 and a standard deviation of 1. It prints the columns being scaled and a confirmation message.

**Output:**
`Scaling columns: ['installment', 'dti', 'fico', 'revol_bal', 'revol_util', 'repayment_velocity_proxy', 'credit_utilization', 'debt_to_income_calc', 'credit_age_years']`
`Scaling complete.`

In [11]:
# 10. Final checks and save
# Drop rows without target if present
if 'target_default' in df.columns:
    df = df.dropna(subset=['target_default'])

# Fill any remaining NaNs with 0 (quick, but you may want a better imputation strategy)
df = df.fillna(0)

# Save processed CSV
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
df.to_csv(OUTPUT_PATH, index=False)
print('Saved processed data to:', OUTPUT_PATH)
print('Final shape:', df.shape)

# Show a quick preview
df.head()

Saved processed data to: /content/loan_data_final_output.csv
Final shape: (9578, 25)


,credit_policy,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,...,credit_utilization,annual_inc,debt_to_income_calc,credit_age_years,has_recent_inq,has_past_delinquency,target_default,purpose_debt_consolidation,purpose_educational,purpose_small_business
0,1,0.001189,2.463099,11.350407,0.998505,0.688825,5639.958333,0.353732,0.182704,0,...,0.145575,85000.000385,1.121950,0.432230,0,0,0,True,False,False
1,1,0.001071,-0.438854,11.082143,0.244540,-0.101303,2760.000000,0.495018,1.030602,0,...,1.025163,65000.000073,-0.585293,-0.721230,0,0,0,False,False,False
2,1,0.001357,0.230708,10.373491,-0.141885,-0.759742,4710.000000,-0.397073,-0.730683,1,...,-0.801949,31999.999943,1.589642,0.059770,1,0,0,True,False,False
3,1,0.001008,-0.757022,11.350407,-0.654697,0.030385,2699.958333,0.496321,0.909966,1,...,0.900019,85000.000385,-1.023161,-0.745277,1,0,0,True,False,False
4,1,0.001426,-1.043992,11.299732,0.343326,-1.154806,4066.000000,-0.360663,-0.251586,0,...,-0.304946,80799.999636,-1.197116,-0.198161,0,1,0,False,False,False


### Explanation for Cell: 10. Final checks and save
This cell performs final data preparation steps: it drops rows where the `target_default` is missing (if it exists) and then fills any remaining `NaN` values in the entire DataFrame with 0 (a simple imputation strategy). Finally, it saves the processed DataFrame to a new CSV file specified by `OUTPUT_PATH`, prints the path and the final shape of the DataFrame, and displays the first 5 rows of the cleaned and processed data.

**Output:**
`Saved processed data to: /content/loan_data_final_output.csv`
`Final shape: (9578, 25)`
It then displays the first 5 rows of the final DataFrame, showing the transformed and scaled values, as well as the new one-hot encoded columns for 'purpose'.

## Next steps
- Run EDA (plots for distributions, correlation matrix, target imbalance).
- Train a baseline model (Logistic Regression) and evaluate AUC-ROC.
- Use MLflow to track experiments.
- Replace simple imputation with more advanced methods if needed.

---
*Notebook generated automatically. Edit paths at the top if you use a different CSV file.*